### Import Packages

In [1]:
import cv2
from google.colab.patches import cv2_imshow
!pip install mediapipe

import mediapipe as mp
from tensorflow.keras.models import load_model
import numpy as np
import copy
import itertools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import tensorflow as tf
tf.__version__

'2.8.0'

### Landmark Detection Functions

In [3]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []
    
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        
        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [4]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))

    
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list


Import the Trained Model

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
model = load_model('/content/drive/My Drive/Colab Notebooks/latestModel')

In [6]:
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

labels_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25,'space':26,'del':27,'nothing':28}


In [7]:
#from google.colab.patches import cv2_imshow
def get_key(val):
    for key, value in labels_dict.items():
        if val == value:
            return key
    return "nothing"

### Setup of Webcam to Accept Live Stream

In [8]:
import base64
import html
import io
import time

from IPython.display import display, Javascript
from google.colab.output import eval_js
import numpy as np
from PIL import Image
import cv2


def start_input():
    js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var modelOut
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       modelOut = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom(label) {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      modelOut = document.createElement('div');
      modelOut.style.fontWeight = 'bold';
      modelOut.style.height = "50px";
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = video.videoWidth;
      captureCanvas.height = video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom(label);
      
      var preShow = Date.now();
      if (label != "") {
        modelOut.innerHTML  = "<H1 style='color:blue;'>Translation: "+label+"</H1>" 
        } else {
          modelOut.innerHTML  = "<H1></H1>"
        }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

    display(js)


def take_photo(label, img_data):
    data = eval_js('takePhoto("{}", "{}")'.format(label, img_data))
    return data

In [9]:
from PIL import Image

def js_reply_to_image(js_reply):
    """
    input: 
          js_reply: JavaScript object, contain image from webcam

    output: 
          open_cv_image
    """
    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes)).convert('RGB')
    open_cv_image = np.array(image_PIL)
    return open_cv_image

In [13]:
#WORKING using colab model
from google.colab import output
from datetime import datetime,timedelta

start_input()
label_html = ""
img_data = ''
count = 0
class_x_1 =[]
tempList =[]
startTime = datetime.now()
    
# Process the model and the image taken above:
while True:
    js_reply = take_photo(label_html, img_data)
    if not js_reply:
        break
    label_html = ""
    redictions=""
    image = js_reply_to_image(js_reply)
    results = hands.process(image)
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks,results.multi_handedness):
            landmark_list = calc_landmark_list(image, hand_landmarks)
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            predictions = model.predict(np.array([pre_processed_landmark_list], dtype=np.float32))
            classes_x=np.argmax(np.squeeze(predictions))
            key = get_key(classes_x)
            
            tempList.append(key)
            if len(set(tempList)) != 1:
                tempList = []
                startTime = datetime.now()
                
            if int((datetime.now() - startTime).total_seconds()) > 1: #hold the hand sign for at-least 1s
                if key:
                    if key == "space":
                        class_x_1.append(" ")
                    elif key == "del":
                        if (len(class_x_1) > 0):
                            class_x_1.pop()
                    elif key != "nothing":
                        class_x_1.append(key)
                tempList = []
                startTime = datetime.now()
            
            
            label_html = ''.join(class_x_1)

<IPython.core.display.Javascript object>

In [15]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 33.4 MB/s 
     |████████████████████████████████| 8.8 MB 61.9 MB/s 
     |████████████████████████████████| 79 kB 9.5 MB/s 
     |████████████████████████████████| 138 kB 77.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [11]:

#Working USING Predict FUNCTION
import os
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = 'sagemaker-tensorflow-serving-2022-05-27-02-43-44-834'
runtime= boto3.client('runtime.sagemaker', 
                     region_name='us-east-2', # make sure to set correct region
                     aws_access_key_id='',
                      aws_secret_access_key='')
while True:
    #success, image = cap.read()
    image = cv2.imread("/content/drive/My Drive/Colab Notebooks/W_test.jpg", cv2.IMREAD_COLOR)
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    predictions=""
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks,results.multi_handedness):
            landmark_list = calc_landmark_list(imgRGB, hand_landmarks)
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            payload = ','.join(str(e) for e in pre_processed_landmark_list)
            response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                   ContentType='text/csv',
                                   Body=payload)
            
            jsonData = json.loads(response['Body'].read().decode())
            classes_x=np.argmax(np.squeeze(jsonData["predictions"]))
            print("\nLabel")
            print(get_key(classes_x))
            print("\n")
            cv2.putText(image, get_key(classes_x),(10,60), cv2.FONT_HERSHEY_PLAIN,3, (255,0,255),4)
    #remove the below break statement when you are ready to make mutiple api calls 
    break
    cv2.imshow("Results", image)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break



Label
W




In [19]:
#  TESTING  Working USING lamda FUNCTION
import json
import csv
import requests
from google.colab import output
from datetime import datetime,timedelta

start_input()
label_html = ""
img_data = ''
count = 0
class_x_1 =[]
tempList =[]
startTime = datetime.now()

URL = 'https://9cmmrs6rwb.execute-api.us-east-2.amazonaws.com/test/predict'
header = {"Content-Type":"application/json"}

while True:
    js_reply = take_photo(label_html, img_data)
    if not js_reply:
        break
    label_html = ""
    image = js_reply_to_image(js_reply)
    results = hands.process(image)
    predictions=""
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks,results.multi_handedness):
            landmark_list = calc_landmark_list(image, hand_landmarks)
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            payload = ','.join(str(e) for e in pre_processed_landmark_list)
            data_payload = '{"data":"'+payload+'"}'
            #print(data_payload)
            request = requests.request("POST",URL, headers=header, data=data_payload)
            #print()

            #jsonData = json.loads(request['Body'].read().decode())
            #classes_x=np.argmax(np.squeeze(jsonData["predictions"]))
            classes_x=np.argmax(np.squeeze(request.json()[0]))
            key = get_key(classes_x)
            
            tempList.append(key)
            if len(set(tempList)) != 1:
                tempList = []
                startTime = datetime.now()
                
            if int((datetime.now() - startTime).total_seconds()) > 1: #hold the hand sign for at-least 1s
                if key:
                    if key == "space":
                        class_x_1.append(" ")
                    elif key == "del":
                        if (len(class_x_1) > 0):
                            class_x_1.pop()
                    elif key != "nothing":
                        class_x_1.append(key)
                tempList = []
                startTime = datetime.now()
            
            
            label_html = ''.join(class_x_1)
            #print("\nLabel")
            #print(key)
            #print("\n")
    #remove the below break statement when you are ready to make mutiple api calls 
    #break


<IPython.core.display.Javascript object>